In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 17.2 MB/s eta 0:00:00a 0:00:01


In [13]:
import cv2
import os
import numpy as np
import torch
from ultralytics import YOLO
from typing import List, Dict

class CombatZoneObjectTracker:
    def __init__(self, model_path: str = 'best.pt', train_data: str = '/kaggle/input/plzzz-bro/data.yaml', epochs: int = 40):
        if not os.path.exists(model_path):
            print("Trained model not found. Starting training...")
            self._train_model(train_data, epochs)
        else:
            print(f"Loading trained model: {model_path}")
        self.detection_model = YOLO(model_path)
        print("YOLO model loaded successfully")
        self.kalman_filters = {}
        self.last_positions = {}
        self.next_id = 0

    def _train_model(self, data_path: str, epochs: int):
        model = YOLO("yolov8n.pt")  # Start with YOLOv8 nano
        model.train(data=data_path, epochs=epochs)
        trained_model_path = "runs/train/exp/weights/best.pt"
        if os.path.exists(trained_model_path):
            print(f"Training complete. Using {trained_model_path}")
            self.detection_model = YOLO(trained_model_path)
        else:
            print("Error: Training failed, model not found.")

    def _initialize_kalman_filter(self):
        kalman = cv2.KalmanFilter(4, 2)
        kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        kalman.transitionMatrix = np.array([
            [1, 0, 1, 0],
            [0, 1, 0, 1],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ], np.float32)
        kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
        kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.00003
        return kalman
    
    def detect_objects(self, frame: np.ndarray) -> List[Dict]:
        results = self.detection_model(frame)
        detected_objects = []
        
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                center_x = (x1 + x2) / 2
                center_y = (y1 + y2) / 2
                detected_objects.append({
                    'bbox': [x1, y1, x2, y2],
                    'confidence': confidence,
                    'center': (center_x, center_y),
                    'class_id': class_id
                })
        return detected_objects
    
    def track_objects(self, detected_objects: List[Dict]) -> List[Dict]:
        tracked_objects = []
        
        for obj in detected_objects:
            center = obj['center']
            tracker_id = self.next_id
            self.next_id += 1
            self.kalman_filters[tracker_id] = self._initialize_kalman_filter()
            kalman = self.kalman_filters[tracker_id]
            kalman.predict()
            measurement = np.array(center, dtype=np.float32)
            kalman.correct(measurement)
            self.last_positions[tracker_id] = center
            tracked_objects.append({
                **obj,
                'tracker_id': tracker_id
            })
        return tracked_objects

def main():
    tracker = CombatZoneObjectTracker(model_path='/kaggle/working/runs/detect/train4/weights/best.pt', train_data='/kaggle/input/plzzz-bro/data.yaml', epochs=40)
    video_path = "/kaggle/input/shobaaa/Indian Army.mp4"
    output_path = "/kaggle/working/combat_zone_tracking.avi"
    
    video_capture = cv2.VideoCapture(video_path)
    
    if not video_capture.isOpened():
        print(f"Error: Unable to open video file: {video_path}")
        return
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        
        objects = tracker.detect_objects(frame)
        tracked_objects = tracker.track_objects(objects)
        
        for obj in tracked_objects:
            x1, y1, x2, y2 = map(int, obj['bbox'])
            confidence = obj['confidence']
            tracker_id = obj['tracker_id']
            class_id = obj['class_id']
            color = ((tracker_id * 50) % 255, (tracker_id * 80) % 255, (tracker_id * 120) % 255)
            label = f"ID: {tracker_id} Conf: {confidence:.2f} Class: {class_id}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        out.write(frame)
    
    video_capture.release()
    out.release()
    print(f"Processing complete. Output saved as {output_path}")

if __name__ == "__main__":
    main()


Loading trained model: /kaggle/working/runs/detect/train4/weights/best.pt
YOLO model loaded successfully

0: 384x640 2 Military Trucks, 10.0ms
Speed: 1.6ms preprocess, 10.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Military Trucks, 7.1ms
Speed: 2.1ms preprocess, 7.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Military Trucks, 6.9ms
Speed: 1.5ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 APCs, 2 Military Trucks, 7.0ms
Speed: 1.4ms preprocess, 7.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Military Trucks, 1 Tank, 6.7ms
Speed: 1.4ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Military Trucks, 6.5ms
Speed: 1.1ms preprocess, 6.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 APCs, 2 Military Trucks, 6.6ms
Speed: 1.3ms preprocess, 6.6ms inf

In [14]:
!zip -r output.zip /kaggle/working/runs/detect/train4

  adding: kaggle/working/runs/detect/train4/ (stored 0%)
  adding: kaggle/working/runs/detect/train4/F1_curve.png (deflated 9%)
  adding: kaggle/working/runs/detect/train4/confusion_matrix.png (deflated 27%)
  adding: kaggle/working/runs/detect/train4/train_batch0.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/train4/val_batch2_pred.jpg (deflated 5%)
  adding: kaggle/working/runs/detect/train4/labels_correlogram.jpg (deflated 34%)
  adding: kaggle/working/runs/detect/train4/train_batch17882.jpg (deflated 8%)
  adding: kaggle/working/runs/detect/train4/val_batch1_labels.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/train4/args.yaml (deflated 53%)
  adding: kaggle/working/runs/detect/train4/events.out.tfevents.1741433536.4eed9ea1fa42.31.3 (deflated 90%)
  adding: kaggle/working/runs/detect/train4/train_batch2.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/train4/results.csv (deflated 61%)
  adding: kaggle/working/runs/detect/train4/val_batch1_pred.jpg (deflated 6